In [ ]:
!python -m pip install --upgrade pip --quiet
!pip install -U awscli --quiet

In [ ]:
from sagemaker import get_execution_role
import sagemaker
import boto3
import os

boto_session = boto3.session.Session()
sm_session = sagemaker.session.Session()
region = boto_session.region_name
sm_client = boto_session.client("sagemaker")
role = get_execution_role()
s3_bucket = sm_session.default_bucket()

In [ ]:
from src.generated.shapes import ContainerDefinition, ProductionVariant, AsyncInferenceConfig, AsyncInferenceOutputConfig, AsyncInferenceClientConfig
# Endpoint Invoking 
from src.generated.resources import Model, EndpointConfig, Endpoint, TrainingJob
from time import gmtime, strftime

key = f'xgboost-iris-{strftime("%H-%M-%S", gmtime())}'
print("key", key)
os.environ['SAGEMAKER_ADMIN_CONFIG_OVERRIDE'] = '/Users/nargokul/workspace/sagemaker-code-gen/sample/sagemaker/2017-07-24/default-configs-inference.json'

In [ ]:
bucket_prefix = "async-inference-demo"
resource_name = "AsyncInferenceDemo-{}-{}"

model_s3_key = f"{bucket_prefix}/demo-xgboost-model.tar.gz"
model_url = f"s3://{s3_bucket}/{model_s3_key}"
print(f"Uploading Model to {model_url}")

with open("model/demo-xgboost-model.tar.gz", "rb") as model_file:
    boto_session.resource("s3").Bucket(s3_bucket).Object(model_s3_key).upload_fileobj(model_file)


from sagemaker import image_uris

# Specify an AWS container image and region as desired
container = image_uris.retrieve(region=region, framework="xgboost", version="0.90-1")

model = Model.create(
    model_name=key,
    primary_container=ContainerDefinition(
        image=container,
        model_data_url=f"s3://{s3_bucket}/{model_s3_key}"
    ),
    execution_role_arn=role,
)

In [ ]:
from datetime import datetime


endpoint_config = EndpointConfig.create(
    endpoint_config_name=key,
    production_variants=[
        ProductionVariant(
            variant_name="variant1",
            model_name=key,
            instance_type='ml.m5.xlarge',
            initial_instance_count=1
        )
    ],
    async_inference_config=AsyncInferenceConfig(
        output_config=AsyncInferenceOutputConfig(s3_output_path=f"s3://{s3_bucket}/{bucket_prefix}/output"),
        client_config=AsyncInferenceClientConfig(
            max_concurrent_invocations_per_instance=4
        )
    )
)


In [ ]:
endpoint = Endpoint.create(endpoint_name=key, endpoint_config_name=key)

In [ ]:
def upload_file(input_location):
    prefix = f"{bucket_prefix}/input"
    return sm_session.upload_data(
        input_location,
        bucket=sm_session.default_bucket(),
        key_prefix=prefix,
        extra_args={"ContentType": "text/libsvm"},
    )

input_1_location = "input/test_point_0.libsvm"
input_1_s3_location = upload_file(input_1_location)



In [ ]:
endpoint = Endpoint.get(endpoint_name=key)
response= endpoint.invoke_async(input_location=input_1_s3_location)